In [ ]:
try:
    print("Importing CUPY...")
    import cupy as cp

    print("done")
except Exception as xEx:
    print(
        "CUDA python module 'cupy' could not be imported.\n"
        "Make sure you have the NVIDIA CUDA toolkit installed and\n"
        "the 'cupy' module. See 'https://cupy.dev/' for more information.\n"
        "Note that if the 'pip' install does not work, try the 'conda' install option.\n\n"
        f"Exception reported:\n{(str(xEx))}"
    )
# endtry

import numpy as np

aA = np.array([1,2,3])
caA = cp.asarray(aA)
fLen = cp.linalg.norm(caA)

print(fLen)



In [ ]:
import os
os.environ["OPENCV_IO_ENABLE_OPENEXR"] = "1"
import cv2

import numpy as np
from pathlib import Path

pathMain = Path(r"[main path]")
pathLocalPos3d = pathMain / "AT_LocalPos3d_Raw"
pathObjectIdx = pathMain / "AT_ObjectIdx_Raw"
pathObjectLoc3d = pathMain / "AT_ObjectLoc3d_Raw"
pathLabel = pathMain / "AT_Label/full_res/SemSeg"

sFrame1:str = "Frame_0010.exr"
sFrame2:str = "Frame_0011.exr"

def LoadImages(_sFrame:str):
    pathImgLocalPos3d = pathLocalPos3d / _sFrame
    pathImgObjectIdx = pathObjectIdx / _sFrame
    pathImgObjectLoc3d = pathObjectLoc3d / _sFrame

    imgLocalPos3d = cv2.imread(
        pathImgLocalPos3d.as_posix(),
        cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH | cv2.IMREAD_UNCHANGED,
    )

    imgObjectIdx = cv2.imread(
        pathImgObjectIdx.as_posix(),
        cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH | cv2.IMREAD_UNCHANGED,
    )

    imgObjectLoc3d = cv2.imread(
        pathImgObjectLoc3d.as_posix(),
        cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH | cv2.IMREAD_UNCHANGED,
    )

    return imgLocalPos3d, imgObjectIdx, imgObjectLoc3d
# enddef

imgPos1, imgObj1, imgLoc1 = LoadImages(sFrame1)
imgPos2, imgObj2, imgLoc2 = LoadImages(sFrame2)

# aOffset = np.array([[[1e4, 1e4, 1e4]]])
# imgPos1 = imgPos1 - aOffset
# imgPos2 = imgPos2 - aOffset

# imgLoc1 = imgLoc1 - aOffset
# imgLoc2 = imgLoc2 - aOffset

iChX: int = 2
iChY: int = 1
iChZ: int = 0

# print(np.min(imgPos1))
# print(np.max(imgPos1))


In [ ]:
iRows, iCols, iChan = imgObj1.shape
# imgObjFlat1 = imgObj1.reshape(-1, iChan)
# aU, aMaskObjIdx1 = np.unique(imgObjFlat1, axis=0, return_inverse=True)
# imgMaskObjIdx1 = aMaskObjIdx1.reshape(iRows, iCols)

imgObjFlat1 = imgObj1.reshape(-1, iChan)
imgObjFlatId1 = imgObjFlat1[:,0] + imgObjFlat1[:,2]

aU, aMaskObjIdx1 = cp.unique(imgObjFlatId1, return_inverse=True)
imgMaskObjIdx1 = aMaskObjIdx1.reshape(iRows, iCols)


In [ ]:
aObjFlat2 = imgObj2.reshape(-1, iChan)
aObjFlatId2 = aObjFlat2[:,0] + aObjFlat2[:,2]
aObjIdxFlat2 = np.ones((iRows * iCols), dtype=int) * -1

for iObjIdx in range(len(aU)):
    aMask = aObjFlatId2 == aU[iObjIdx].item()
    aObjIdxFlat2[aMask] = iObjIdx
# endfor

imgMaskObjIdx2 = aObjIdxFlat2.reshape(iRows, iCols)


In [ ]:
aInvalidIdx = np.argwhere(aU < 1.0)
for iIdx in aInvalidIdx:
    imgMaskObjIdx1[imgMaskObjIdx1 == iIdx.item()] = -1
    imgMaskObjIdx2[imgMaskObjIdx2 == iIdx.item()] = -1
# endfor


In [ ]:
import matplotlib.pyplot as plt
from scipy import spatial
from scipy import interpolate

iObjIdx = 2
# plt.imshow(imgInv)
imgMask1 = imgMaskObjIdx1 == iObjIdx
imgObjPos1 = np.zeros((iRows, iCols, 4), dtype=float)
imgObjPos1[imgMask1, 0:3] = imgPos1[imgMask1, :]
imgObjPos1[imgMask1, 3] = 1.0
plt.imshow(imgObjPos1)


In [ ]:

imgMask2 = np.all(imgObj2 == aU[iObjIdx][np.newaxis, np.newaxis, :], axis=2)
imgObjPos2 = np.zeros((iRows, iCols, 4), dtype=float)
imgObjPos2[imgMask2, 0:3] = imgPos2[imgMask2, :]
imgObjPos2[imgMask2, 3] = 1.0
plt.imshow(imgObjPos2)


In [ ]:
print(cp.cuda.runtime.getDeviceCount())


In [ ]:
import math

def SphericalToCartesian(_imgPos: np.ndarray, *, _tRgbIdx: tuple[int, int, int] = (0, 1, 2)) -> np.ndarray:

    imgPos = np.zeros_like(_imgPos)

    imgRad = _imgPos[:, :, _tRgbIdx[0]]
    imgTheta = _imgPos[:, :, _tRgbIdx[1]]
    imgPhi = _imgPos[:, :, _tRgbIdx[2]] - math.pi
    imgSinTheta = np.sin(imgTheta)

    imgPos[:, :, _tRgbIdx[0]] = imgRad * imgSinTheta * np.cos(imgPhi)
    imgPos[:, :, _tRgbIdx[1]] = imgRad * imgSinTheta * np.sin(imgPhi)
    imgPos[:, :, _tRgbIdx[2]] = imgRad * np.cos(imgTheta)

    return imgPos

# enddef


In [ ]:
# cfdImage = cp.cuda.texture.ChannelFormatDescriptor(32, 32, 32, 0, cp.cuda.runtime.cudaChannelFormatKindFloat)
import sys
from pathlib import Path

pathKernel = Path.cwd() / "Dev-EvalFlow-v3.cu"
sKernelCode = pathKernel.read_text()

iThreadCnt = 32
tiSearchRadiusXY = (200, 200)

iPosRows, iPosCols, iPosChanCnt = imgPos1.shape
tiSizeXY = (iPosCols, iPosRows)
tiStartXY = (2175, 1018)
tiRangeXY = (100, 10)

# Full image
# tiStartXY = (0, 0)
# tiRangeXY = (iPosCols, iPosRows)

tiRangeXY = tuple(tiRangeXY[i] if tiStartXY[i] + tiRangeXY[i] <= tiSizeXY[i] else tiSizeXY[i] - tiStartXY[0] for i in range(2))

tiBlockDimXY = (tiRangeXY[0] // iThreadCnt + (1 if tiRangeXY[0] % iThreadCnt > 0 else 0), tiRangeXY[1])

iIdxCols = tiRangeXY[0]
iIdxRows = tiRangeXY[1]
iIdxChan = 5

iPosRowStride = iPosCols * iPosChanCnt
iIdxRowStride = iIdxChan * iIdxCols

iSubPixChanCnt = 4
iSubPixRowStride = iSubPixChanCnt * iIdxCols

sFuncFlowExp = (f"EvalFlow<{tiStartXY[0]}, {tiStartXY[1]}, "
                f"{tiRangeXY[0]}, {tiRangeXY[1]}, "
                f"{tiSizeXY[0]}, {tiSizeXY[1]}, "
                f"{tiSearchRadiusXY[0]}, {tiSearchRadiusXY[1]}, "
                f"{iPosChanCnt}, {iPosRowStride}, "
                f"{iIdxChan}, {iIdxRowStride}, "
                f"{iSubPixChanCnt}, {iSubPixRowStride}>")
                
modKernel = cp.RawModule(code=sKernelCode, options=("-std=c++11",), name_expressions=[sFuncFlowExp])
# modKernel.compile(log_stream=sys.stdout)
kernFlow = modKernel.get_function(sFuncFlowExp)

caIdxMapXY = cp.ones((iIdxRows, iIdxCols, iIdxChan), dtype=cp.int32)
caIdxMapXY *= -1

caSubPixMapXY = cp.full((iIdxRows, iIdxCols, iSubPixChanCnt), cp.nan, dtype=cp.float32)

imgPosConv1 = SphericalToCartesian(imgPos1, _tRgbIdx=(2,1,0))
imgPosConv2 = SphericalToCartesian(imgPos2, _tRgbIdx=(2,1,0))

caPos1 = cp.asarray(imgPosConv1, dtype=cp.float32)
caPos2 = cp.asarray(imgPosConv2, dtype=cp.float32)
caMaskObjIdx1 = cp.asarray(imgMaskObjIdx1, dtype=cp.int32)
caMaskObjIdx2 = cp.asarray(imgMaskObjIdx2, dtype=cp.int32)

kernFlow(tiBlockDimXY, (iThreadCnt,), (caPos1, caPos2, caMaskObjIdx1, caMaskObjIdx2, caIdxMapXY, caSubPixMapXY))

# cp.cuda.Stream.null.synchronize()

aIdxMapXY = cp.asnumpy(caIdxMapXY)
aSubPixMapXY = cp.asnumpy(caSubPixMapXY)

cv2.imwrite((pathMain / "PosConv1.exr").as_posix(), imgPosConv1)
cv2.imwrite((pathMain / "PosConv2.exr").as_posix(), imgPosConv2)


In [ ]:
iSelObjIdx = 3
# Select object id and those points that have a valid map
aMaskObjIdx = np.logical_and(aIdxMapXY[:, :, 0] == iSelObjIdx, aIdxMapXY[:, :, 3] >= 0)

aFlatIdxXY = aIdxMapXY[aMaskObjIdx][:, 1:3].astype(float)
aFlatMapIdxXY = aSubPixMapXY[aMaskObjIdx][:, 0:2]
aFlowLines = np.stack((aFlatIdxXY, aFlatMapIdxXY), axis=1)

# Plot flow image
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

clnFlow = LineCollection(aFlowLines[::10], linewidths=1.0, cmap="jet", alpha=0.5)

figFlow, axFlow = plt.subplots()
# axFlow.imshow(imgObjPos1, alpha=0.5)
# axFlow.imshow(imgObjPos2, alpha=0.5)

axFlow.add_collection(clnFlow)
axFlow.set_xlim(max(0, tiStartXY[0] - tiSearchRadiusXY[0]), min(tiStartXY[0] + tiRangeXY[0] + tiSearchRadiusXY[0], tiSizeXY[0]))
axFlow.set_ylim(min(tiStartXY[1] + tiRangeXY[1] + tiSearchRadiusXY[1], tiSizeXY[1]), max(0, tiStartXY[1] - tiSearchRadiusXY[1]))
axFlow.margins(0.1)


In [ ]:
aMaskValidFlow = aIdxMapXY[:, :, 3] >= 0
aValidIdxXY = aIdxMapXY[aMaskValidFlow][:, 1:3]
aValidIdxPos = aValidIdxXY[:,1] * iPosCols + aValidIdxXY[:, 0]

aFlowXY = aSubPixMapXY[aMaskValidFlow][:, 2:4]
aObjIdx = aIdxMapXY[aMaskValidFlow][:, 0].astype(float)

imgValidMap = np.zeros((iPosRows, iPosCols), dtype=bool)
aValidMapFlat = imgValidMap.flatten()
aValidMapFlat[aValidIdxPos] = True
imgValidMap = aValidMapFlat.reshape(iPosRows, iPosCols)

imgFlow = np.zeros((iPosRows, iPosCols, 4), dtype=np.float32)
imgFlow[imgValidMap, iChX] = aFlowXY[:,0]
imgFlow[imgValidMap, iChY] = aFlowXY[:,1]
imgFlow[imgValidMap, iChZ] = aObjIdx
imgFlow[imgValidMap, 3] = 1.0

pathFlow = pathMain / "Flow.exr"
cv2.imwrite(pathFlow.as_posix(), imgFlow)
